In [1]:
! pip install diffusers transformers accelerate torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 14.6 MB/s eta 0:00:00


In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-video-diffusion-img2vid")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/6.10G [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from PIL import Image

# Load the image using PIL
image_path = "/content/127552249.png"
image = Image.open(image_path)

In [ ]:
prompt = "/content/127552249.png"
video_frames = pipe(prompt, num_inference_steps=40, height=320, width=576, num_frames=24).frames
video_path = export_to_video(video_frames)

In [ ]:
from flask import Flask, render_template, request, send_file
from diffusers import pipe, export_to_video

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/generate_video', methods=['POST'])
def generate_video():
    if request.method == 'POST':
        prompt = request.form['prompt']

        # Call the pipe function with the provided prompt
        video_frames = pipe(prompt, num_inference_steps=40, height=320, width=576, num_frames=24).frames

        # Call the export_to_video function to export frames to a video file
        video_path = export_to_video(video_frames)

        # Return the path to the generated video file
        return send_file(video_path, as_attachment=True)

if __name__ == '__main__':
    app.run(debug=True)
